In [32]:
import numpy as np
import pandas as pd

In [58]:
from torch import nn,rand


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        # self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1)
        # self.relu1 = nn.ReLU()
        # self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1)
        # self.relu2 = nn.ReLU()
        # self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2)
        # output = nn.Linear(in_features=14,output)

        # output size = (N - F + padding) / stride + 1
         
        self.clf = nn.Sequential(
         # output size = (28 + 2(pad) - 3 ) / 1 + 1 = 28
         nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1),
         nn.ReLU(inplace=True),
         # output size = (28 - 3) / 1 + 1 = 26
         nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1),
         nn.ReLU(inplace=True),
         # output size = 26 / 2 = 13
         nn.MaxPool2d(kernel_size=2,stride=2),
        )

        self.fc = nn.Sequential(
            nn.Linear(in_features=64*13*13, out_features=100),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=100,out_features=10),
            nn.Softmax()
        )
    
    def forward(self,input):
        output = self.clf(input)
        output = output.view(output.size()[0],-1)
        output = self.fc(output)
        
        return output

        

In [34]:
input_tensor = rand(1,1,28,28)

model = ConvNet()

output = model(input_tensor)

print('output type:', type(output), 'output:', output)

output type: <class 'torch.Tensor'> output: tensor([[0.0901, 0.1023, 0.1001, 0.1067, 0.0984, 0.1043, 0.1117, 0.0981, 0.0896,
         0.0987]], grad_fn=<SoftmaxBackward0>)


/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [38]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import torch
from sklearn.model_selection import train_test_split

def get_preprocessed_data(images, labels):

    images = np.array(images/255.0, dtype=np.float32)
    lables = np.array(labels, dtype=np.float32)

    return images, labels

# 0~1사이값 float32로 변경하는 함수 호출 한 뒤 OHE 적용
def get_preprocessed_ohe(images, labels):
    images, labels = get_preprocessed_data(images,labels)
    oh_labels = nn.functional.one_hot(labels,len(torch.unique(labels)))
    return images, oh_labels

# 학습 검증 테스트 데이터 세트에 전처리 및 ohe 적용한 뒤 반환
def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    train_images, train_oh_lables = get_preprocessed_ohe(train_images,train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images,test_labels)

    tr_images, val_images, train_oh_lables, val_oh_labels = train_test_split(train_images, train_oh_lables,test_size=valid_size,random_state=random_state)

    return (tr_images, train_oh_lables), (val_images, val_oh_labels), (test_images, test_oh_labels)

In [65]:
transform = ToTensor()

train_data = MNIST(root='data',download=True,transform=transform,train=True)
test_data = MNIST(root='data',download=True,transform=transform,train=False)

train_images, train_labels = train_data.data.float()/255.0 , train_data.targets
test_images, test_labels = test_data.data.float()/255.0 , test_data.targets

print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(tr_images, train_oh_lables), (val_images, val_oh_labels), (test_images, test_oh_labels) = get_train_valid_test_set(train_images,train_labels,test_images,test_labels,valid_size=0.15,random_state=2021)

train_data_loader =  torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=False)
test_data_loader = torch.utils.data.DataLoader(test_data,batch_size=128,shuffle=True)

torch.Size([60000, 28, 28]) torch.Size([60000]) torch.Size([10000, 28, 28]) torch.Size([10000])


In [66]:
import torchvision.transforms.functional as TF

model = ConvNet()

optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_function = torch.nn.CrossEntropyLoss()

def train(epoch):    
    model.train()
    for num_iter, (images, labels) in enumerate(train_data_loader):
        print(num_iter,"/",len(train_data_loader))

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

@torch.no_grad()
def eval(epoch):
    model.eval()

    test_loss = 0.0
    correct = 0.0

    for (images, labels) in test_data_loader:

        outputs = model(images)
        loss = loss_function(outputs, labels)

        test_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum()

        print('Test set: Epoch: {}, Average loss: {:.4f}, Accuracy: {:.4f}'.format(
        epoch,
        test_loss / len(test_data_loader.dataset),
        correct.float() / len(test_data_loader.dataset)
    ))


for epoch in range(1,2):
    train(epoch)
    acc = eval(epoch)

@torch.no_grad()
def test():
    model.eval()
    
    outputs = model(test_images)
    loss = loss_function(outputs,test_labels)
    

    



0 / 469
1 / 469
2 / 469
3 / 469
4 / 469
5 / 469
6 / 469
7 / 469
8 / 469
9 / 469
10 / 469
11 / 469
12 / 469
13 / 469
14 / 469
15 / 469
16 / 469
17 / 469
18 / 469
19 / 469
20 / 469
21 / 469
22 / 469
23 / 469
24 / 469
25 / 469
26 / 469
27 / 469
28 / 469
29 / 469
30 / 469
31 / 469
32 / 469
33 / 469
34 / 469
35 / 469
36 / 469
37 / 469
38 / 469
39 / 469
40 / 469
41 / 469
42 / 469
43 / 469
44 / 469
45 / 469
46 / 469
47 / 469
48 / 469
49 / 469
50 / 469
51 / 469
52 / 469
53 / 469
54 / 469
55 / 469
56 / 469
57 / 469
58 / 469
59 / 469
60 / 469
61 / 469
62 / 469
63 / 469
64 / 469
65 / 469
66 / 469
67 / 469
68 / 469
69 / 469
70 / 469
71 / 469
72 / 469
73 / 469
74 / 469
75 / 469
76 / 469
77 / 469
78 / 469
79 / 469
80 / 469
81 / 469
82 / 469
83 / 469
84 / 469
85 / 469
86 / 469
87 / 469
88 / 469
89 / 469
90 / 469
91 / 469
92 / 469
93 / 469
94 / 469
95 / 469
96 / 469
97 / 469
98 / 469
99 / 469
100 / 469
101 / 469
102 / 469
103 / 469
104 / 469
105 / 469
106 / 469
107 / 469
108 / 469
109 / 469
110 / 469


In [67]:
print(test_labels.shape)

torch.Size([10000])
